In [7]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_community.document_loaders import UnstructuredPDFLoader

In [8]:
# pdf file location
localpath = "C:/Drive D/Documents/Why should I trust you.pdf"

# read the file from the location
try:
    pdf_loader = UnstructuredPDFLoader(file_path = localpath)
    pdf_data = pdf_loader.load()
except Exception as e:
    print(e)

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 20)
chunks = text_splitter.split_documents(pdf_data)

In [22]:
url = "http://localhost:6333/"
qdrant = Qdrant.from_documents(
    chunks,
    OllamaEmbeddings(model = "nomic-embed-text", show_progress = True),
    url=url,
    prefer_grpc=False,
    collection_name="local-qdrant-rag",
)

OllamaEmbeddings: 100%|██████████| 38/38 [03:46<00:00,  5.96s/it]


In [23]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [24]:
# llm from google
local_model = "gemma:2b"
llm = ChatOllama(model = local_model)

In [25]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [27]:
retriever = MultiQueryRetriever.from_llm(
    qdrant.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [28]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
chain.invoke(input(""))

 what this document is about?


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


'This document is about the purpose and techniques of explaining and selecting instances for a machine learning model, particularly when dealing with trust-related tasks.'